In [1]:
import lyricwikia
import pandas as pd

df = pd.read_csv("pop_playlist_project.csv")
df.columns = ['Spotify_URI','TrackName','ArtistName','AlbumName','DiscNum','TrackNum','TrackDur','AddedBy','AddedAt']
lyrics = ""

for i in range(len(df)):
    try:
        lyrics_temp = lyricwikia.get_lyrics(df.ArtistName[i], df.TrackName[i])
        lyrics=lyrics+lyrics_temp
    except:
        continue
#print(lyrics)

Now that she's back in the atmosphere
With drops of Jupiter in her hair, hey
She acts like summer and walks like rain
Reminds me that there's time to change, hey
Since the return of her stay on the moon
She listens like spring, and she talks like June, hey, hey

Tell me, did you sail across the sun?
Did you make it to the Milky Way to see the lights all faded
And that Heaven is overrated?
Tell me, did you fall from shooting star
One without a permanent scar?
And did you miss me while you were looking for yourself out there?

Now that she's back from that soul vacation
Tracing her way through the constellation, hey, mm
She checks out Mozart while she does Tae-Bo
Reminds me that there's a room to grow, hey, yeah

Now that she's back in the atmosphere
I'm afraid that she might think of me as plain ol' Jane
Told a story about a man who was too afraid to fly so he never did land

But tell me, did the wind sweep you off your feet?
Did you finally get the chance to dance along the light of da

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

c:\users\caleb\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
raw_text = lyrics.lower()
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ",n_vocab)

Total Characters:  744681
Total Vocab:  70


In [4]:
seq_length = len(df)
dataX = []
dataY = []
for i in range(0,n_chars - seq_length,1):
    seq_in = raw_text[i:i +seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ",n_patterns)

Total Patterns:  743832


In [6]:
X = numpy.reshape(dataX,(n_patterns,seq_length,1))
X = X/float(n_vocab)
y=np_utils.to_categorical(dataY)

In [10]:
model = Sequential()
#256
model.add(LSTM(30,input_shape=(X.shape[1],X.shape[2]),return_sequences = True))
model.add(Dropout(0.2))
#256
model.add(LSTM(15))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [ ]:
model.fit(X,y)
#epochs=10,batch_size=128

Epoch 1/1
348640/743832 [=============>................] - ETA: 2:18:46 - loss: 2.9349

In [ ]:
int_to_char = dict((i,c) for i,c in enumerate(chars))
initial_text="My lyrics for my random pop song"
pattern=[char_to_int[char] for char in initial_text]
pattern=pattern[0:seq_length]

text_gen=""

for i in range(1000):
    x = numpy.reshape(pattern,(1,len(pattern),1))
    x = x/float(n_vocab)
    prediction = model.